In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time
import requests
import time as t
from commonregex import CommonRegex
from streetaddress import StreetAddressFormatter, StreetAddressParser
import pandas as pd

In [2]:
!which chromedriver

/opt/homebrew/bin/chromedriver


In [16]:
executable_path = {'executable_path': '/opt/homebrew/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [17]:
url = "https://www.golflink.com/golf-courses"
State = "CA" 

City = "Palm Desert"

st_city_url = url + "/" + State.lower() + "/" + City.replace(" ","-").lower() + "/"
st_city_url

'https://www.golflink.com/golf-courses/ca/palm-desert/'

In [18]:
browser.visit(st_city_url)

In [19]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [23]:
course_dict = {}
dict_list = []

courses = soup.find_all("div", class_="resultContainer")
for course in courses:
    course_dict = {}
    name = course.find("a", class_="fly")
    address_blurb = course.find("div", class_ = "textcontainer")
    text_add_blur = address_blurb.text
    clean_add_blurb = " ".join(text_add_blur.split())
    course_dict["Name"] = name.text.split(',')[0]
    course_dict["Course Name"] = name.text.split(',')[1]
    course_dict["Holes"] = re.search(r'.*holes', clean_add_blurb)[0]
    try:
        pub_priv = re.findall('(Private|Public|Municipal)', clean_add_blurb)[0]
        course_dict["Public/Private"] = pub_priv
    except:
        course_dict["Public/Private"] = "N/A"
    course_dict["Phone"] = re.findall("\(\w{3}\) \w{3}-\w{4}",clean_add_blurb)[0]

    phone = re.findall("\(\w{3}\) \w{3}-\w{4}",clean_add_blurb)
    add = clean_add_blurb.rpartition(phone[0])[0]
    course_dict["Address"] = (re.split('Write a Review Check Tee Times |Write a Review ', add)[-1])
    course_dict["Blurb"] = clean_add_blurb
    dict_list.append(course_dict)


In [24]:
dict_list

[{'Name': 'Marrakesh Country Club',
  'Course Name': ' Marrakesh Course',
  'Holes': '18 holes',
  'Public/Private': 'Private',
  'Phone': '(760) 568-2660',
  'Address': '47000 Marrakesh Dr Palm Desert, CA 92260-5805 ',
  'Blurb': '18 holes over 3,614 yards with a par of 60 (Private) Write a Review 47000 Marrakesh Dr Palm Desert, CA 92260-5805 (760) 568-2660 1 mile from the center of Palm Desert. Write a Review'},
 {'Name': 'Shadow Mountain Golf Club',
  'Course Name': ' Shadow Mountain Course',
  'Holes': '18 holes',
  'Public/Private': 'Public',
  'Phone': '(760) 346-8242',
  'Address': '73800 Ironwood St Palm Desert, CA 92260-5531 ',
  'Blurb': '18 holes over 5,441 yards with a par of 70 (Public) Write a Review Check Tee Times 73800 Ironwood St Palm Desert, CA 92260-5531 (760) 346-8242 1 mile from the center of Palm Desert. Write a Review Check Tee Times'},
 {'Name': 'The Golf Center At Palm Desert',
  'Course Name': ' Palm Desert Course',
  'Holes': '9 holes',
  'Public/Private': '

In [25]:
df = pd.DataFrame.from_dict(dict_list)
df

,Name,Course Name,Holes,Public/Private,Phone,Address,Blurb
0,Marrakesh Country Club,Marrakesh Course,18 holes,Private,(760) 568-2660,"47000 Marrakesh Dr Palm Desert, CA 92260-5805","18 holes over 3,614 yards with a par of 60 (Pr..."
1,Shadow Mountain Golf Club,Shadow Mountain Course,18 holes,Public,(760) 346-8242,"73800 Ironwood St Palm Desert, CA 92260-5531","18 holes over 5,441 yards with a par of 70 (Pu..."
2,The Golf Center At Palm Desert,Palm Desert Course,9 holes,Public,(760) 347-3484,"74945 Sheryl Ave Palm Desert, CA 92260-2073",9 holes over 914 yards with a par of 27 (Publi...
3,Portola Country Club,Portola Course,18 holes,Private,(760) 346-5481,"42500 Portola Ave Palm Desert, CA 92260-1998","18 holes over 2,134 yards with a par of 54 (Pr..."
4,Ironwood Country Club,North Course,18 holes,Private,(760) 766-1062,"73735 Irontree Dr Palm Desert, CA 92260-6910","18 holes over 6,122 yards with a par of 72 (Pr..."
...,...,...,...,...,...,...,...
95,The Madison Club,Madison Course,18 holes,Private,(760) 391-4500,"53035 Meriwether Way La Quinta, CA 92253-5535","18 holes over 7,426 yards with a par of 72 (Pr..."
96,Mission Hills North Gary Player Signature Course,Gary Player Signature Course,18 holes,Public,(760) 699-0111,"70705 Ramon Rd Rancho Mirage, CA 92270-5602","18 holes over 7,020 yards with a par of 72 (Pu..."
97,PGA West Private Golf Courses,Tom Weiskop Course,18 holes,Private,(760) 564-3914,"55955 Pga Blvd La Quinta, CA 92253-5677","18 holes over 7,164 yards with a par of 72 (Pr..."
98,PGA West Private Golf Courses,Jack Nicklaus Course,18 holes,Private,(760) 564-3914,"55955 Pga Blvd La Quinta, CA 92253-5677","18 holes over 6,909 yards with a par of 72 (Pr..."


In [26]:
df.to_csv("Data_Scraped/Palm_Desert_courses.csv", index=False)